In [1]:
bike = sqlContext.read.format('csv').options(header='true', nullValue='\\N').load('/mnt/bike/fargo')

In [2]:
# Build Trip Duration
from pyspark.sql import functions as F
bike = bike.withColumn("Checkout", F.to_timestamp(F.concat(F.col("CheckoutDate"), F.lit(" "), F.col("CheckoutTime")))).withColumn("Return", F.to_timestamp(F.concat(F.col("ReturnDate"), F.lit(" "), F.col("ReturnTime"))))
bike = bike.withColumn("Duration", F.col("Return").cast("long") - F.col("Checkout").cast("long"))

display(bike.take(10))

# RiderType,BikeNumber,CheckoutStation,ReturnStation,CheckoutDate,ReturnDate,CheckoutTime,ReturnTime,Checkout,Return,Duration
Student,2376,University Village,Wallman Wellness Center,2015-06-11,2015-06-11,12:46:54,12:55:33,2015-06-11T12:46:54.000+0000,2015-06-11T12:55:33.000+0000,519
Student,2360,Wallman Wellness Center,University Village,2015-06-17,2015-06-17,16:36:42,16:44:40,2015-06-17T16:36:42.000+0000,2015-06-17T16:44:40.000+0000,478
Student,2400,University Village,Wallman Wellness Center,2015-06-16,2015-06-16,12:40:17,12:49:01,2015-06-16T12:40:17.000+0000,2015-06-16T12:49:01.000+0000,524
Student,2400,Wallman Wellness Center,University Village,2015-06-16,2015-06-16,16:33:24,16:42:01,2015-06-16T16:33:24.000+0000,2015-06-16T16:42:01.000+0000,517
Student,2385,High Rise Complex,Wallman Wellness Center,2015-06-17,2015-06-17,12:53:07,16:36:48,2015-06-17T12:53:07.000+0000,2015-06-17T16:36:48.000+0000,13421
Student,2378,University Village,Sanford Medical Center,2015-06-18,2015-06-18,19:20:23,19:37:21,2015-06-18T19:20:23.000+0000,2015-06-18T19:37:21.000+0000,1018
Student,2378,Sanford Medical Center,University Village,2015-06-18,2015-06-18,19:38:42,20:21:51,2015-06-18T19:38:42.000+0000,2015-06-18T20:21:51.000+0000,2589
Student,2358,University Village,University Village,2015-06-28,2015-06-28,19:58:05,20:34:19,2015-06-28T19:58:05.000+0000,2015-06-28T20:34:19.000+0000,2174
Student,2371,Fercho YMCA,Fercho YMCA,2015-06-28,2015-06-28,16:59:38,17:15:22,2015-06-28T16:59:38.000+0000,2015-06-28T17:15:22.000+0000,944
Student,2371,Wallman Wellness Center,Fercho YMCA,2015-06-28,2015-06-28,16:38:23,16:59:28,2015-06-28T16:38:23.000+0000,2015-06-28T16:59:28.000+0000,1265


In [3]:
from pyspark.sql.functions import mean, min, max

bike.select([mean('Duration'), min('Duration'), max('Duration')]).show()

+------------------+-------------+-------------+
 avg(Duration)|min(Duration)|max(Duration)|
+------------------+-------------+-------------+
1052.2633480183808| 0| 1120124|
+------------------+-------------+-------------+

In [4]:
# Stolen blatentnly from https://stackoverflow.com/a/51633111/404006 
# This Season Data
from pyspark import *
from pyspark.sql import functions as F
aggregatebike = bike.where(F.year("CheckoutDate") == 2017).groupBy(F.year("CheckoutDate").alias("year"), "BikeNumber").agg(F.mean("Duration"), F.count("Duration").alias("count")).sort(["year", "count"], ascending=False)
display(aggregatebike)

year,BikeNumber,avg(Duration),count
2017,2380,881.212201591512,1508
2017,2440,1540.915208613728,1486
2017,2366,865.1632653061224,1421
2017,2452,677.2749822820695,1411
2017,2364,547.277304964539,1410
2017,2455,1086.682176091625,1397
2017,2447,937.7175627240143,1395
2017,2367,677.2057761732852,1385
2017,2437,771.3572463768116,1380
2017,2379,878.945335276968,1372


In [5]:
#Last Season Data
aggregatebike = bike.where(F.year("CheckoutDate") == 2016).groupBy(F.year("CheckoutDate").alias("year"), "BikeNumber").agg(F.mean("Duration"), F.count("Duration").alias("count")).sort(["year", "count"], ascending=False)
display(aggregatebike)

year,BikeNumber,avg(Duration),count
2016,2413,803.9266365688487,1772
2016,2419,913.3885681293302,1732
2016,2447,833.5224852071005,1690
2016,2449,957.3777777777777,1620
2016,2374,842.5315034310668,1603
2016,2456,943.5994781474234,1533
2016,2439,1218.1384105960265,1510
2016,2416,1177.932486631016,1496
2016,2425,1041.8526244952893,1486
2016,2442,851.1629680998614,1442


In [6]:
#All Time Data
aggregatebike = bike.groupBy("BikeNumber").agg(F.mean("Duration"), F.count("Duration").alias("count")).sort("count", ascending=False)
display(aggregatebike)

BikeNumber,avg(Duration),count
2447,930.094084393297,4953
2372,1017.6247020585049,4615
2414,927.4662372278987,4502
2437,1041.2247491638796,4485
2419,846.7262708052182,4446
2410,1114.7338801549329,4389
2379,979.6565656565657,4356
2413,1117.8235970561177,4348
2449,991.5512702078522,4330
2442,843.7254765225476,4302


In [7]:
#All Time Last, First Ride
sortbike = bike.select("BikeNumber", "CheckoutDate").where(F.col("BikeNumber") == 2447)
df1 = sortbike.sort("CheckoutDate", ascending=True).first()
df2 = sortbike.sort("CheckoutDate", ascending=False).first()

#Last Season Last First Ride
sortbike = bike.select("BikeNumber", "CheckoutDate").where(F.col("BikeNumber") == 2413)
df3 = sortbike.sort("CheckoutDate", ascending=True).first()
df4 = sortbike.sort("CheckoutDate", ascending=False).first()

#This Season Last First Ride
sortbike = bike.select("BikeNumber", "CheckoutDate").where(F.col("BikeNumber") == 2380)
df5 = sortbike.sort("CheckoutDate", ascending=True).first()
df6 = sortbike.sort("CheckoutDate", ascending=False).first()

display([df1, df2, df3, df4, df5, df6])



BikeNumber,CheckoutDate
2447,2015-03-15
2447,2017-11-02
2413,2015-03-19
2413,2017-11-01
2380,2015-03-30
2380,2017-10-31
